Setup environment and load the base PyPSA-Earth network for a specified country.


In [4]:
# %% [markdown]
# Sequential experiment: influence of transformer reactance defaults
# (x_pu_default, x_abs_default) on load shedding and congestion.

# %%
import os
import sys
import time
import logging
import warnings
from os.path import join
from pathlib import Path
from itertools import product

import numpy as np
import pandas as pd
import pypsa

# ---------- Logging & warnings ----------
warnings.filterwarnings("ignore")
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)
logger = logging.getLogger(__name__)

# ---------- Project paths ----------
parent_dir = Path(os.getcwd()).parents[0]          # project/
sys.path.append(str(parent_dir))
from src.paths import all_dirs
dirs = all_dirs()

OUTDIR = Path(dirs["results/experiments"])
OUTDIR.mkdir(parents=True, exist_ok=True)
OUT_CSV = OUTDIR / "experiment_x_defaults_results.csv"

# ---------- Network path ----------
net_dir = Path(dirs["data/processed/networks"])
candidate = net_dir / "network_base_filled.nc"
assert candidate.is_file(), f"Could not find a network to load at {candidate}"
NETWORK_PATH = str(candidate)
logger.info(f"Base network: {NETWORK_PATH}")

# ---------- Helpers ----------
def ensure_load_shedding(n: pypsa.Network, ls_cost=100e4):
    if not hasattr(n, "carriers") or n.carriers is None:
        n.carriers = pd.DataFrame(index=pd.Index([], name="carrier"))
    if "load_shedding" not in n.carriers.index:
        n.carriers.loc["load_shedding", ["co2_emissions", "color", "nice_name"]] = [0.0, "#000000", "Load Shedding"]

    need = []
    for b in n.buses.index:
        name = f"LS_{b}"
        if name not in n.generators.index:
            need.append((name, b))
    if need:
        n.madd(
            "Generator",
            names=[nm for nm, _ in need],
            bus=[b for _, b in need],
            carrier="load_shedding",
            p_nom=1e6,
            marginal_cost=float(ls_cost)
        )

def month_window(n: pypsa.Network, hours=24*30) -> pd.Index:
    if len(n.snapshots) == 0:
        raise ValueError("Network has no snapshots.")
    return n.snapshots[:hours] if len(n.snapshots) >= hours else n.snapshots

def patch_transformers_defaults(n: pypsa.Network, x_pu_default: float, x_abs_default: float,
                                r_pu_default: float = 0.01, r_abs_default: float = 0.01):
    if n.transformers.empty:
        return
    tr = n.transformers.copy()
    for col in ["r_pu", "x_pu"]:
        if col not in tr.columns:
            tr[col] = np.nan
    if hasattr(n, "transformer_types") and not n.transformer_types.empty and "type" in tr.columns:
        for col in ["r_pu", "x_pu"]:
            if col in n.transformer_types.columns:
                filled = tr[col].isna() | (tr[col] == 0)
                tr.loc[filled, col] = tr.loc[filled, "type"].map(n.transformer_types[col])
    tr.loc[tr["r_pu"].isna() | (tr["r_pu"] == 0), "r_pu"] = float(r_pu_default)
    tr.loc[tr["x_pu"].isna() | (tr["x_pu"] == 0), "x_pu"] = float(x_pu_default)
    if "r" not in tr.columns: tr["r"] = np.nan
    if "x" not in tr.columns: tr["x"] = np.nan
    tr.loc[tr["r"].isna() | (tr["r"] == 0), "r"] = float(r_abs_default)
    tr.loc[tr["x"].isna() | (tr["x"] == 0), "x"] = float(x_abs_default)
    if "s_nom" in tr.columns:
        tr.loc[tr["s_nom"].fillna(0) <= 0, "s_nom"] = 1.0
    n.transformers.loc[tr.index, tr.columns] = tr

def compute_kpis(n: pypsa.Network, solved_snaps: pd.Index) -> dict:
    total_load_ts = n.loads_t.p_set.loc[solved_snaps].sum(axis=1) if not n.loads_t.p_set.empty else pd.Series(0.0, index=solved_snaps)
    demand_energy_mwh = float(total_load_ts.sum())

    ls_idx = n.generators.index[n.generators["carrier"] == "load_shedding"]
    if len(ls_idx):
        ls_ts = n.generators_t.p.loc[solved_snaps, ls_idx].sum(axis=1)
        ls_energy_mwh = float(ls_ts.sum())
        ls_peak_mw = float(ls_ts.max())
    else:
        ls_energy_mwh = 0.0
        ls_peak_mw = 0.0

    ls_percent = (ls_energy_mwh / demand_energy_mwh * 100.0) if demand_energy_mwh > 0 else 0.0

    if len(n.lines):
        s_nom = n.lines["s_nom"].replace({0: np.nan})
        line_loading = n.lines_t.p0.loc[solved_snaps].abs().div(s_nom, axis=1)
        max_line_loading_pu = float(line_loading.max().max())
    else:
        max_line_loading_pu = np.nan

    if len(n.transformers):
        s_nom_tr = n.transformers["s_nom"].replace({0: np.nan})
        tr_loading = n.transformers_t.p0.loc[solved_snaps].abs().div(s_nom_tr, axis=1)
        max_trafo_loading_pu = float(tr_loading.max().max())
    else:
        max_trafo_loading_pu = np.nan

    peak_load_mw = float(total_load_ts.max()) if len(total_load_ts) else 0.0

    curtail_mwh = np.nan
    try:
        if hasattr(n.generators_t, "p_max_pu") and not n.generators_t.p_max_pu.empty:
            avail = (n.generators_t.p_max_pu.loc[solved_snaps] * n.generators.p_nom.values).sum(axis=1)
            out = n.generators_t.p.loc[solved_snaps].sum(axis=1)
            curtail_ts = (avail - out).clip(lower=0)
            curtail_mwh = float(curtail_ts.sum())
    except Exception:
        pass

    return {
        "demand_energy_mwh": demand_energy_mwh,
        "load_shed_energy_mwh": ls_energy_mwh,
        "load_shed_percent": ls_percent,
        "load_shed_peak_mw": ls_peak_mw,
        "max_line_loading_pu": max_line_loading_pu,
        "max_trafo_loading_pu": max_trafo_loading_pu,
        "peak_load_mw": peak_load_mw,
        "curtailment_energy_mwh": curtail_mwh,
    }

def run_one(network_path: str, x_pu_default: float, x_abs_default: float,
            solver_name: str, solver_options: dict, hours: int) -> dict:
    case_id = f"xpu={x_pu_default:.2f}_xabs={x_abs_default:.2f}"
    t0 = time.time()
    try:
        n = pypsa.Network(network_path)
        ensure_load_shedding(n)
        patch_transformers_defaults(
            n,
            x_pu_default=float(x_pu_default),
            x_abs_default=float(x_abs_default),
            r_pu_default=0.01,
            r_abs_default=0.01,
        )
        snaps = month_window(n, hours=hours)
        n.optimize(snapshots=snaps, solver_name=solver_name, solver_options=solver_options)
        kpis = compute_kpis(n, solved_snaps=snaps)
        status, msg = "ok", ""
    except Exception as e:
        status, msg = "fail", repr(e)
        kpis = {k: np.nan for k in [
            "demand_energy_mwh", "load_shed_energy_mwh", "load_shed_percent",
            "load_shed_peak_mw", "max_line_loading_pu", "max_trafo_loading_pu",
            "peak_load_mw", "curtailment_energy_mwh"
        ]}
    elapsed = time.time() - t0
    return {
        "case": case_id,
        "x_pu_default": float(x_pu_default),
        "x_abs_default": float(x_abs_default),
        "status": status,
        "message": msg,
        "runtime_s": elapsed,
        **kpis
    }

# ---------- Experiment grid ----------
VALUES = np.linspace(0.0, 0.25, 4)   # quick test; use 6 for full sweep
GRID = list(product(VALUES, VALUES))

# ---------- Solver ----------
SOLVER = "highs"
SOLVER_OPTS = {"threads": max(1, os.cpu_count() // 2), "presolve": "on"}  # use a few threads sequentially

# ---------- Window ----------
MONTH_HOURS = 24*15   # set 720 for full month

# ---------- Resume support ----------
done = set()
rows = []
if OUT_CSV.exists():
    try:
        prev = pd.read_csv(OUT_CSV)
        # identify finished cases by their (x_pu_default, x_abs_default)
        for _, r in prev.iterrows():
            done.add((round(float(r["x_pu_default"]), 6), round(float(r["x_abs_default"]), 6)))
        rows = prev.to_dict(orient="records")
        logger.info(f"Resuming: {len(done)} cases already completed.")
    except Exception:
        logger.warning("Could not read previous results; starting fresh.")

# ---------- Run sequentially ----------
if __name__ == "__main__":
    total = len(GRID)
    logger.info(f"Starting sequential experiment on {total} cases.")

    for i, (xpu, xabs) in enumerate(GRID, 1):
        key = (round(float(xpu), 6), round(float(xabs), 6))
        if key in done:
            logger.info(f"[{i}/{total}] Skip (already done): xpu={xpu:.2f}, xabs={xabs:.2f}")
            continue

        logger.info(f"[{i}/{total}] Running: xpu={xpu:.2f}, xabs={xabs:.2f}")
        res = run_one(NETWORK_PATH, xpu, xabs, SOLVER, SOLVER_OPTS, MONTH_HOURS)
        rows.append(res)

        # checkpoint after each case
        pd.DataFrame(rows).sort_values(["x_pu_default", "x_abs_default"]).reset_index(drop=True).to_csv(OUT_CSV, index=False)
        if res["status"] != "ok":
            logger.warning(f"Case failed: {res['case']} → {res['message']}")
        else:
            logger.info(f"Done {res['case']} in {res['runtime_s']:.1f}s; LS={res['load_shed_percent']:.2f}%")

    df = pd.DataFrame(rows).sort_values(["x_pu_default", "x_abs_default"]).reset_index(drop=True)
    ok = (df["status"] == "ok").sum()
    fail = (df["status"] != "ok").sum()
    logger.info(f"Finished: {ok} ok / {fail} failed. Results → {OUT_CSV}")

    if ok:
        keep = df[df["status"] == "ok"]
        best = keep.nsmallest(5, "load_shed_percent")[["x_pu_default","x_abs_default","load_shed_percent","max_line_loading_pu","max_trafo_loading_pu","runtime_s"]]
        worst = keep.nlargest(5, "load_shed_percent")[["x_pu_default","x_abs_default","load_shed_percent","max_line_loading_pu","max_trafo_loading_pu","runtime_s"]]
        logger.info("\nTop-5 lowest load shedding (%):\n" + best.to_string(index=False))
        logger.info("\nTop-5 highest load shedding (%):\n" + worst.to_string(index=False))


2025-11-12 16:08:50: Base network: c:\Repositories\Repos\pypsa-earth-project\EcuadorElectricGrid\data\processed\networks\network_base_filled.nc
2025-11-12 16:08:50: Starting sequential experiment on 16 cases.
2025-11-12 16:08:50: [1/16] Running: xpu=0.00, xabs=0.00
2025-11-12 16:08:51: Imported network network_base_filled.nc has buses, carriers, generators, lines, loads, transformers
2025-11-12 16:08:51:  Solve problem using Highs solver
2025-11-12 16:08:51: Solver options:
 - threads: 8
 - presolve: on
2025-11-12 16:08:52: Writing objective.


Writing continuous variables.: 100%|██████████| 3/3 [00:00<00:00, 12.85it/s]

2025-11-12 16:08:53:  Writing time: 1.56s


2025-11-12 16:08:58:  Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 132840 primals, 313200 duals
Objective: 3.99e+10
Solver model: available
Solver message: Optimal

2025-11-12 16:08:58: The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Line-fix-s-lower, Line-fix-s-upper, Transformer-fix-s-lower, Transformer-fix-s-upper, Kirchhoff-Voltage-Law were not assigned to the network.
2025-11-12 16:08:58: Done xpu=0.00_xabs=0.00 in 7.7s; LS=4.10%
2025-11-12 16:08:58: [2/16] Running: xpu=0.00, xabs=0.08
2025-11-12 16:08:58: Imported network network_base_filled.nc has buses, carriers, generators, lines, loads, transformers
2025-11-12 16:08:59:  Solve problem using Highs solver
2025-11-12 16:08:59: Solver options:
 - threads: 8
 - presolve: on
2025-11-12 16:08:59: Writing objective.


Writing continuous variables.: 100%|██████████| 3/3 [00:00<00:00, 13.40it/s]

...

2025-11-12 16:11:11:  Writing time: 2.09s


2025-11-12 16:11:16:  Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 132840 primals, 313200 duals
Objective: 3.99e+10
Solver model: available
Solver message: Optimal

2025-11-12 16:11:16: The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Line-fix-s-lower, Line-fix-s-upper, Transformer-fix-s-lower, Transformer-fix-s-upper, Kirchhoff-Voltage-Law were not assigned to the network.
2025-11-12 16:11:16: Done xpu=0.25_xabs=0.17 in 9.2s; LS=4.10%
2025-11-12 16:11:16: [16/16] Running: xpu=0.25, xabs=0.25
2025-11-12 16:11:16: Imported network network_base_filled.nc has buses, carriers, generators, lines, loads, transformers
2025-11-12 16:11:17:  Solve problem using Highs solver
2025-11-12 16:11:17: Solver options:
 - threads: 8
 - presolve: on
2025-11-12 16:11:17: Writing objective.


Writing continuous variables.: 100%|██████████| 3/3 [00:00<00:00, 12.93it/s]

2025-11-12 16:11:19:  Writing time: 1.56s


2025-11-12 16:11:24:  Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 132840 primals, 313200 duals
Objective: 3.99e+10
Solver model: available
Solver message: Optimal

2025-11-12 16:11:24: The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Line-fix-s-lower, Line-fix-s-upper, Transformer-fix-s-lower, Transformer-fix-s-upper, Kirchhoff-Voltage-Law were not assigned to the network.
2025-11-12 16:11:24: Done xpu=0.25_xabs=0.25 in 8.0s; LS=4.10%
2025-11-12 16:11:24: Finished: 16 ok / 0 failed. Results → c:\Repositories\Repos\pypsa-earth-project\EcuadorElectricGrid\results\experiments\experiment_x_defaults_results.csv
2025-11-12 16:11:24: 
Top-5 lowest load shedding (%):
 x_pu_default  x_abs_default  load_shed_percent  max_line_loading_pu  max_trafo_loading_pu  runtime_s
     0.000000       0.000000             4.1027                  0.7              0.810932   7.728755
     0.000000       0.083333             4.1027             